In [1]:
import itertools
import pathlib
import pyam # version: 1.4.1.dev15+g3563b43

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

C:\Users\gidden\Miniconda3\envs\pyam\Lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


<IPython.core.display.Javascript object>

# Load Data

In [2]:
data_path = pathlib.Path('../data')

In [3]:
paper_data = pd.concat([
    pd.read_csv(data_path / 'gidden_et_al_2022_ar6_reanalysis_data.csv'),
    pd.read_csv(data_path / 'additional_analysis_data.csv'),
])
df = pyam.IamDataFrame(paper_data)
df.load_meta(data_path / 'gidden_et_al_2022_ar6_reanalysis_meta.csv')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading meta indicators for 914 out of 1871 scenarios


In [4]:
categories_to_temp = {'C1': '1.5C', 'C3': '2C'}
df.meta['Temperature'] = df.meta['Category'].replace(categories_to_temp)

In [5]:
kyoto_gases_variable = 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases'

# Figure 2A

In [ ]:
variables = [
    kyoto_gases_variable,
]
quantiles = (0.1, 0.25, 0.5, 0.75, 0.9)
year_start, year_end = 2010, 2100

data = pyam.concat([
    (
        df
        .filter(region='World', variable=variable, year=range(year_start, year_end + 1))
        .filter(Category=category)
        .convert_unit('Mt CO2-equiv/yr', to='Gt CO2-equiv/yr', factor=1e-3)
        .compute.quantiles(quantiles)
        .rename(model={'Quantiles': temp})
    ) for (category, temp), variable in itertools.product(categories_to_temp.items(), variables)
])

for variable in variables:
    data.append(
        df
        .filter(region='World', variable=variable, year=range(year_start, year_end + 1))
        .filter(Policy_category_name='P1b: Current policies')
        .convert_unit('Mt CO2-equiv/yr', to='Gt CO2-equiv/yr', factor=1e-3)
        .compute.quantiles(quantiles)
        .rename(model={'Quantiles': 'Current Policies'}),
        inplace=True
    )

In [ ]:
# taken from p17, row 'Global emissions, including the impact of overachievement of NDCs *' of den Elzen 2022 SI
targets = pd.DataFrame([
    ('Current Policies (den Elzen 2022, adj.)', 56.826, 52.592, 59.552, 'dimgrey', 0),
    ('Conditional NDCs (den Elzen 2022, adj.)', 50.700, 48.542, 52.202, 'plum', -0.35),
    ('Unconditional NDCs (den Elzen 2022, adj.)', 52.284, 50.059, 54.136, 'thistle', 0.35),
], columns=['label', 'med', 'min', 'max', 'color', 'jitter'])

target_2015_value = 51.181 # to adjust for differences in LULUCF definitions, we take an offset of values used in den Elzen per conversations with Niklas Forsell
this_study_2015_value = (
    data
    .filter(variable=kyoto_gases_variable, year=2015, scenario='0.5')
    .timeseries()
    .mean()
    .iloc[0]
)
target_offset = target_2015_value - this_study_2015_value
target_2015_value, this_study_2015_value, target_offset

In [ ]:
color_map = {
    'Current Policies': pyam.plotting.PYAM_COLORS["AR6-IP-CurPol"],
    '2C': pyam.plotting.PYAM_COLORS["AR6-C1"],
    '1.5C': pyam.plotting.PYAM_COLORS["AR6-C3"], 
}
rc = pyam.run_control()
#rc.store['color']['model'] = {}
rc.update({'color': {'model': color_map}})

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
(
    data
    .filter(variable=kyoto_gases_variable, year=range(2015, 2051))
    .filter(scenario=['*0.25*', '*0.75*'])
    .plot
    .line(color='model', fill_between={'alpha': 0.5}, ax=ax, alpha=0.1)
)
(
    data
    .filter(variable=kyoto_gases_variable, year=range(2015, 2051))
    .filter(scenario='0.5')
    .plot
    .line(color='model', ax=ax, cmap='Set2')
)
for i, row in targets.iterrows():
    ax.scatter(2030 + row['jitter'], row['med'] - target_offset, label=row['label'], c=row['color'])
    ax.errorbar(2030 + row['jitter'], row['med'] - target_offset, 
                yerr=np.array([[row['med'] - row['min'], row['max'] - row['med']],]).reshape(2, 1), 
                color=row['color'])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[6:], labels[6:])
ax.set_title('Total GHGs adjusted to NGHGIs compared with NDCs')
ax.set_xlabel('')
ax.set_ylim(0, 70)

fig.savefig('./files/fig3a.pdf', bbox_inches='tight')
fig.savefig('./files/fig3a.jpg', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
(
    data
    .filter(variable=kyoto_gases_variable, year=range(2015, 2101))
    .filter(scenario=['*0.25*', '*0.75*'])
    .plot
    .line(color='model', fill_between=True, ax=ax, alpha=0.5)
)
(
    data
    .filter(variable=kyoto_gases_variable, year=range(2015, 2101))
    .filter(scenario='0.5')
    .plot
    .line(color='model', ax=ax, cmap='Set2')
)
for i, row in targets.iterrows():
    ax.scatter(2030 + row['jitter'], row['med'] - target_offset, label=row['label'], c=row['color'])
    ax.errorbar(2030 + row['jitter'], row['med'] - target_offset, 
                yerr=np.array([[row['med'] - row['min'], row['max'] - row['med']],]).reshape(2, 1), 
                color=row['color'])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[6:], labels[6:])
ax.set_title('Total GHGs adjusted to NGHGIs compared with NDCs')
ax.set_xlabel('')
ax.set_ylim(-10, 80)

## Gap quantification

In [ ]:
med_target = targets.set_index('label')['med'] - target_offset
med_target

In [ ]:
pathways = (
    data
    .filter(variable=kyoto_gases_variable, year=2030, scenario=['*0.5*'])
    .timeseries()
    [2030]
    .unstack('model')
    .reset_index(drop=True)
)
pathways

In [ ]:
gap = pd.DataFrame({c: med_target - float(pathways[c]) for c in pathways})
# gap.loc[-2:, 'Current Policies'] = np.nan
gap

In [ ]:
# values in 2030
estimates_den_elzen = {
    '2C': 41.5, # 66% chance
    '1.5C': 24.7,
}
for key, value in estimates_den_elzen.items():
    gap[f'{key} (den Elzen 2022, adj.)'] = med_target - (value - target_offset)

In [ ]:
gap

In [ ]:
gap.columns

In [ ]:
gap = gap.rename(columns= {
    '1.5C': '1.5C (this study)',
    '2C': '2C (this study)',
})

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
order = ['2C (this study)', '2C (den Elzen 2022, adj.)', '1.5C (this study)', '1.5C (den Elzen 2022, adj.)']
gap[order].loc[gap.index[::-1]].plot.barh(cmap='Paired', ax=ax)
ax.legend(**pyam.plotting.OUTSIDE_LEGEND['right'])
ax.set_ylabel('')
ax.set_xlabel('Gt CO2/yr')
ax.set_title('Emission gap between targets, current policies, and pledges')

In [ ]:
(12.7 - 10.8) / 10.8 # 2C

In [ ]:
(26.4 - 27.6) / 27.6

# Figure 2B

In [ ]:
def make_gap_data(df, v, categories_to_temp):
    x = (
        df
        .filter(variable=v, year=[2030])
        .filter(Category=categories_to_temp.keys())
        .as_pandas()
        .replace({'Category': categories_to_temp})
    )
    idx = ['model', 'scenario', 'region', 'year', 'Category']
    x = x.set_index(idx)['value'] 
    return x

def make_gap_plot(data, drop={}, ax=None, palette=None):
    data = data.reset_index().dropna()
    for col, value in drop.items():
        data = data[data[col] != value]
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 5))
    ax.axhline(0, c='k', ls='--', alpha=0.5)
    palette = palette or {"2C": "#fc8d59", "1.5C":"#91bfdb"}
    sns.boxplot(
        x="region", y="value", hue="Category",
        data=data, 
        palette=palette, ax=ax, showfliers=False, whis=0,
    )
    return ax

In [ ]:
# change in emissions in negative, gap is positive
a = 'Change in AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases from 2020'
b = 'Change in AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only from 2020'
adj = -1 * make_gap_data(df, a, categories_to_temp)
orig = -1 * make_gap_data(df, b, categories_to_temp)

In [ ]:
(
    df
    .filter(variable=[a, b], year=range(2020, 2031, 5))
    .subtract(a, b, name='Change in Indirect Component of Flux', ignore_units='Mt CO2-equiv/yr')
    .filter(Category=['C1', 'C3'])
    .filter(region='World')
    .plot.line(color='Category')
)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))

make_gap_plot(adj-orig, ax=ax, palette=color_map)#, drop={'region': 'World'}) # World values distort plot
ax.set_xlabel('')
ax.set_ylabel(r"$\Delta_{NGHGI} - \Delta_{Model}$ (Mt CO2-equiv/yr)", fontsize=16)
ax.set_title(
    'Change in 2030 regional emission reductions between \n'
    'pathways with and without the NGHGI correction',
    fontsize=16,
)
plt.xticks(rotation=45, fontsize=14)


Note that this calculation differs from that in Figure 2. Here we look at the relative change in emission reductions between both frameworks.

$$\Delta_{model} = E^{GHG}_{model, 2030} - E^{GHG}_{model, 2020}$$
$$\Delta_{NGHGI} = E^{GHG}_{NGHGI, 2030} - E^{GHG}_{NGHGI, 2020}$$

$$metric = 1 - \frac{\Delta^{2030-2020}_{NGHGI}}{\Delta^{2030-2020}_{Model}}$$

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))

make_gap_plot(100 * ((adj / orig) - 1), ax=ax, palette=color_map)
ax.set_xlabel('')
ax.set_ylabel(r"$\Delta_{NGHGI} / \Delta_{Model} - 1$ (%)", fontsize=16)

ax.set_title(
    'Change in 2020-2030 emissions reductions using\n'
    'NGHGI conventions compared with model conventions',
    fontsize=16,
)
plt.xticks(rotation=45, fontsize=14)

fig.savefig('./files/fig3b.pdf', bbox_inches='tight')